# Phase 3: Model Evaluation — Qini Curves & Calibration Analysis

## Uplift Modeling Project — "The Persuadable Hunter"

This notebook covers:
1. **Qini Curve** — The standard metric for uplift model evaluation
2. **Random Baseline** — How much better are we than random targeting?
3. **AUUC Calculation** — Area Under the Uplift Curve
4. **Calibration Analysis** — Investigating the Decile 9 anomaly
5. **Model Interpretation** — What the curves tell us about our model

---

### Key Insight from Phase 2

Gemini (Senior Mentor) identified an interesting pattern in the Decile Analysis:

| Decile | Control % | Treatment % | Observed Lift |
|--------|-----------|-------------|---------------|
| 8 | 0.32% | 1.53% | **+1.22pp** (best!) |
| 9 | **1.29%** | 1.53% | +0.24pp (anomaly) |
| 10 | 0.93% | 1.88% | +0.95pp |

**Decile 9 has the highest Control conversion rate** → These are "Sure Things" who buy regardless of email. The model confused high baseline conversion with high persuadability.

---

## 1. Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.integrate import trapezoid  # For AUUC calculation (np.trapz deprecated)
import joblib

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("Libraries loaded successfully!")

In [ ]:
# Load test results with uplift scores from Phase 2
results_df = pd.read_csv('../data/test_results_with_uplift.csv')

print(f"Test Results Shape: {results_df.shape[0]:,} rows × {results_df.shape[1]} columns")
print(f"\nColumns: {list(results_df.columns)}")
print(f"\nUplift Score Stats:")
print(results_df['uplift_score'].describe())

In [ ]:
# Extract key columns
y_true = results_df['y_true'].values          # Actual conversion (0/1)
treatment = results_df['treatment'].values    # Treatment assignment (0/1)
uplift_scores = results_df['uplift_score'].values  # Predicted uplift

print(f"Test set summary:")
print(f"  Total customers: {len(y_true):,}")
print(f"  Treated: {treatment.sum():,} ({treatment.mean()*100:.1f}%)")
print(f"  Control: {(1-treatment).sum():,} ({(1-treatment.mean())*100:.1f}%)")
print(f"  Conversions: {y_true.sum():,} ({y_true.mean()*100:.2f}%)")

---

## 2. Understanding the Qini Curve

### What is a Qini Curve?

The Qini curve answers: **"If I target the top X% of customers by uplift score, how many *incremental* conversions do I get?"**

**Key insight**: We can only measure incremental conversions on **treated vs. control** groups, not on individuals.

### The Algorithm

1. Rank all customers by uplift score (highest first)
2. For each % of population targeted:
   - Count conversions in the **treated** subset
   - Count conversions in the **control** subset (scaled to same size)
   - Incremental = Treated conversions - Scaled control conversions
3. Plot cumulative incremental conversions

### Interpretation

- **Good model**: Curve rises steeply, then flattens (captured persuadables early)
- **Random model**: Diagonal line
- **Bad model**: Below the diagonal

In [ ]:
def calculate_qini_curve(y_true, treatment, uplift_scores, n_bins=100):
    """
    Calculate Qini curve data.
    
    Returns:
        percentages: % of population targeted
        qini_values: Cumulative incremental conversions
        random_values: Random baseline values
    
    BUGFIX (Codex review): Ensure endpoint pct=1.0 is always included.
    """
    # Sort by uplift score (descending)
    sorted_idx = np.argsort(uplift_scores)[::-1]
    y_sorted = y_true[sorted_idx]
    t_sorted = treatment[sorted_idx]
    
    n = len(y_true)
    n_treated = treatment.sum()
    n_control = n - n_treated
    
    # Calculate at each point
    percentages = []
    qini_values = []
    random_values = []
    
    # Total conversions for random baseline endpoint
    total_treated_conversions = (y_true * treatment).sum()
    total_control_conversions = (y_true * (1 - treatment)).sum()
    
    # Pre-calculate final qini for random baseline
    scaled_control_total = total_control_conversions * (n_treated / n_control)
    final_qini = total_treated_conversions - scaled_control_total
    
    step = max(1, n // n_bins)
    
    for i in range(1, n + 1, step):
        pct = i / n
        
        # Subset up to position i
        y_subset = y_sorted[:i]
        t_subset = t_sorted[:i]
        
        # Count conversions in treatment and control
        n_t_subset = t_subset.sum()
        n_c_subset = i - n_t_subset
        
        conversions_t = (y_subset * t_subset).sum()
        conversions_c = (y_subset * (1 - t_subset)).sum()
        
        # Scale control to match treatment size
        if n_c_subset > 0 and n_t_subset > 0:
            scaled_control = conversions_c * (n_t_subset / n_c_subset)
            qini = conversions_t - scaled_control
        else:
            qini = 0
        
        # Random baseline: linear from 0 to final qini
        random_qini = pct * final_qini
        
        percentages.append(pct)
        qini_values.append(qini)
        random_values.append(random_qini)
    
    # BUGFIX: Ensure we include the start point (0, 0)
    if percentages[0] != 0:
        percentages.insert(0, 0)
        qini_values.insert(0, 0)
        random_values.insert(0, 0)
    
    # BUGFIX (Codex): Ensure we include the endpoint (1.0, final_qini)
    if percentages[-1] != 1.0:
        # Calculate final point at 100% population
        n_t_full = t_sorted.sum()
        n_c_full = n - n_t_full
        conv_t_full = (y_sorted * t_sorted).sum()
        conv_c_full = (y_sorted * (1 - t_sorted)).sum()
        
        if n_c_full > 0 and n_t_full > 0:
            scaled_control_full = conv_c_full * (n_t_full / n_c_full)
            qini_full = conv_t_full - scaled_control_full
        else:
            qini_full = final_qini
        
        percentages.append(1.0)
        qini_values.append(qini_full)
        random_values.append(final_qini)
    
    return np.array(percentages), np.array(qini_values), np.array(random_values)

print("Qini curve function defined (with endpoint bugfix)!")

In [ ]:
# Calculate Qini curve
percentages, qini_values, random_values = calculate_qini_curve(
    y_true, treatment, uplift_scores, n_bins=100
)

print(f"Qini curve calculated with {len(percentages)} points")
print(f"Endpoints: pct[0]={percentages[0]:.2f}, pct[-1]={percentages[-1]:.2f}")
print(f"\nFinal Qini value (100% targeted): {qini_values[-1]:.2f} incremental conversions")
print(f"Random baseline endpoint: {random_values[-1]:.2f}")

---

## 3. Plotting the Qini Curve with Random Baseline

In [ ]:
# Plot Qini Curve
fig, ax = plt.subplots(figsize=(12, 8))

# Model curve
ax.plot(percentages * 100, qini_values, 'b-', linewidth=2.5, label='T-Learner Model')

# Random baseline (CLEARLY MARKED as requested by Gemini)
ax.plot(percentages * 100, random_values, 'r--', linewidth=2, label='Random Targeting (Baseline)')

# Fill the area between (model gain)
ax.fill_between(percentages * 100, random_values, qini_values, 
                alpha=0.3, color='green', label='Model Gain over Random')

ax.set_xlabel('% of Population Targeted', fontsize=14)
ax.set_ylabel('Cumulative Incremental Conversions', fontsize=14)
ax.set_title('Qini Curve: T-Learner Uplift Model Evaluation', fontsize=16, fontweight='bold')
ax.legend(loc='upper left', fontsize=12)
ax.grid(True, alpha=0.3)

# Add annotations
ax.axhline(y=0, color='gray', linestyle='-', linewidth=0.5)

# Mark key points
for pct in [0.1, 0.3, 0.5]:
    idx = np.argmin(np.abs(percentages - pct))
    ax.scatter([percentages[idx]*100], [qini_values[idx]], color='blue', s=100, zorder=5)
    gain_over_random = qini_values[idx] - random_values[idx]
    ax.annotate(f'{pct*100:.0f}%: +{gain_over_random:.1f} vs random', 
                xy=(percentages[idx]*100, qini_values[idx]),
                xytext=(percentages[idx]*100 + 5, qini_values[idx] + 2),
                fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('../data/qini_curve.png', dpi=150, bbox_inches='tight')
plt.show()

---

## 4. AUUC Calculation (Area Under Uplift Curve)

In [ ]:
def calculate_auuc(percentages, qini_values, random_values):
    """
    Calculate Area Under the Uplift Curve.
    
    Returns:
        auuc: Total area under model curve
        auuc_random: Area under random baseline
        auuc_gain: Gain over random (the metric we care about)
        normalized_auuc: AUUC normalized by random (>1 = better than random)
    """
    # Use trapezoidal rule for integration (scipy.integrate.trapezoid)
    auuc = trapezoid(qini_values, percentages)
    auuc_random = trapezoid(random_values, percentages)
    auuc_gain = auuc - auuc_random
    
    # Normalized: how much better than random?
    if auuc_random != 0:
        normalized_auuc = auuc / auuc_random
    else:
        normalized_auuc = float('inf') if auuc > 0 else 0
    
    return auuc, auuc_random, auuc_gain, normalized_auuc

auuc, auuc_random, auuc_gain, normalized_auuc = calculate_auuc(
    percentages, qini_values, random_values
)

print("AUUC (Area Under Uplift Curve) Analysis")
print("=" * 60)
print(f"Model AUUC:      {auuc:.4f}")
print(f"Random AUUC:     {auuc_random:.4f}")
print(f"AUUC Gain:       {auuc_gain:.4f}")
print(f"Normalized AUUC: {normalized_auuc:.2f}x (vs random baseline)")
print(f"\nInterpretation: Model is {(normalized_auuc-1)*100:.1f}% better than random targeting")

---

## 5. Cumulative Uplift Curve (Alternative Visualization)

In [ ]:
def calculate_cumulative_uplift_curve(y_true, treatment, uplift_scores, n_bins=10):
    """
    Calculate observed uplift at each decile.
    This is what we'll use to check the Decile 9 anomaly.
    """
    # Create deciles
    df = pd.DataFrame({
        'y_true': y_true,
        'treatment': treatment,
        'uplift_score': uplift_scores
    })
    
    # Assign deciles (1-10, where 10 = highest uplift)
    df['decile'] = pd.qcut(df['uplift_score'], q=n_bins, labels=False, duplicates='drop') + 1
    
    # Calculate metrics per decile
    decile_metrics = []
    
    for decile in sorted(df['decile'].unique()):
        subset = df[df['decile'] == decile]
        
        treated = subset[subset['treatment'] == 1]
        control = subset[subset['treatment'] == 0]
        
        if len(treated) > 0 and len(control) > 0:
            treated_rate = treated['y_true'].mean()
            control_rate = control['y_true'].mean()
            observed_uplift = treated_rate - control_rate
            predicted_uplift = subset['uplift_score'].mean()
        else:
            treated_rate = control_rate = observed_uplift = predicted_uplift = 0
        
        decile_metrics.append({
            'decile': decile,
            'n_customers': len(subset),
            'n_treated': len(treated),
            'n_control': len(control),
            'treated_rate': treated_rate,
            'control_rate': control_rate,
            'observed_uplift': observed_uplift,
            'predicted_uplift': predicted_uplift
        })
    
    return pd.DataFrame(decile_metrics)

decile_df = calculate_cumulative_uplift_curve(y_true, treatment, uplift_scores, n_bins=10)
print("Decile Analysis:")
print("=" * 90)
decile_df

---

## 6. Calibration Analysis: The Decile 9 Anomaly

Gemini identified that Decile 9 has:
- **Highest control rate** (Sure Things who buy anyway)
- **Low observed lift** (little incremental value from email)

This is a calibration issue: the model confuses "high baseline conversion" with "high persuadability".

In [ ]:
# Calibration Plot: Predicted vs Observed Uplift by Decile
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Left: Predicted vs Observed Uplift
ax = axes[0]
x = decile_df['decile']
ax.bar(x - 0.2, decile_df['predicted_uplift'] * 100, width=0.4, 
       label='Predicted Uplift', color='#3498db', alpha=0.8)
ax.bar(x + 0.2, decile_df['observed_uplift'] * 100, width=0.4, 
       label='Observed Uplift', color='#2ecc71', alpha=0.8)
ax.axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
ax.set_xlabel('Uplift Decile (1=Lowest, 10=Highest)', fontsize=12)
ax.set_ylabel('Uplift (percentage points)', fontsize=12)
ax.set_title('Model Calibration: Predicted vs Observed Uplift', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.set_xticks(x)

# Highlight Decile 9 anomaly
decile_9 = decile_df[decile_df['decile'] == 9].iloc[0]
ax.annotate('Decile 9 Anomaly:\nHigh predicted,\nlow observed uplift', 
            xy=(9, decile_9['observed_uplift']*100),
            xytext=(7, decile_9['predicted_uplift']*100 + 0.5),
            arrowprops=dict(arrowstyle='->', color='red', lw=2),
            fontsize=10, color='red', fontweight='bold')

# Right: Control Rate by Decile (The Root Cause)
ax = axes[1]
colors = ['#e74c3c' if d == 9 else '#3498db' for d in decile_df['decile']]
bars = ax.bar(decile_df['decile'], decile_df['control_rate'] * 100, color=colors, alpha=0.8)
ax.set_xlabel('Uplift Decile (1=Lowest, 10=Highest)', fontsize=12)
ax.set_ylabel('Control Conversion Rate (%)', fontsize=12)
ax.set_title('Root Cause: Control Rate by Decile', fontsize=14, fontweight='bold')
ax.set_xticks(decile_df['decile'])

# Highlight Decile 9
ax.annotate('Decile 9 has HIGHEST\ncontrol rate = "Sure Things"', 
            xy=(9, decile_9['control_rate']*100),
            xytext=(6.5, decile_9['control_rate']*100 + 0.2),
            arrowprops=dict(arrowstyle='->', color='red', lw=2),
            fontsize=10, color='red', fontweight='bold')

plt.tight_layout()
plt.savefig('../data/calibration_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Cumulative Uplift Curve by Decile (to check for Decile 9 dip)
fig, ax = plt.subplots(figsize=(12, 7))

# Calculate cumulative observed uplift (from Decile 10 down to 1)
decile_df_sorted = decile_df.sort_values('decile', ascending=False)
decile_df_sorted['cumulative_treated_conversions'] = (
    decile_df_sorted['treated_rate'] * decile_df_sorted['n_treated']
).cumsum()
decile_df_sorted['cumulative_control_conversions'] = (
    decile_df_sorted['control_rate'] * decile_df_sorted['n_control']
).cumsum()
decile_df_sorted['cumulative_n_treated'] = decile_df_sorted['n_treated'].cumsum()
decile_df_sorted['cumulative_n_control'] = decile_df_sorted['n_control'].cumsum()

# Scale control to treatment size
decile_df_sorted['scaled_control'] = (
    decile_df_sorted['cumulative_control_conversions'] * 
    decile_df_sorted['cumulative_n_treated'] / 
    decile_df_sorted['cumulative_n_control'].replace(0, 1)
)
decile_df_sorted['cumulative_incremental'] = (
    decile_df_sorted['cumulative_treated_conversions'] - 
    decile_df_sorted['scaled_control']
)

# Percentage of population from high to low uplift
decile_df_sorted['cum_pct'] = decile_df_sorted['n_customers'].cumsum() / decile_df_sorted['n_customers'].sum()

# Plot
ax.plot(decile_df_sorted['cum_pct'] * 100, decile_df_sorted['cumulative_incremental'], 
        'b-o', linewidth=2, markersize=8, label='Model')

# Random baseline
max_incr = decile_df_sorted['cumulative_incremental'].iloc[-1]
ax.plot([0, 100], [0, max_incr], 'r--', linewidth=2, label='Random Baseline')

# Mark each decile
for _, row in decile_df_sorted.iterrows():
    d = int(row['decile'])
    color = 'red' if d == 9 else 'blue'
    fontweight = 'bold' if d == 9 else 'normal'
    ax.annotate(f'D{d}', xy=(row['cum_pct']*100, row['cumulative_incremental']),
                xytext=(row['cum_pct']*100 + 2, row['cumulative_incremental'] + 0.5),
                fontsize=9, color=color, fontweight=fontweight)

ax.set_xlabel('% of Population Targeted (High to Low Uplift)', fontsize=12)
ax.set_ylabel('Cumulative Incremental Conversions', fontsize=12)
ax.set_title('Cumulative Uplift by Decile — Checking for Decile 9 Dip', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/cumulative_uplift_by_decile.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Marginal uplift by decile (to see if Decile 9 dips)
fig, ax = plt.subplots(figsize=(12, 6))

colors = ['#e74c3c' if d == 9 else ('#27ae60' if u > 0 else '#95a5a6') 
          for d, u in zip(decile_df['decile'], decile_df['observed_uplift'])]

bars = ax.bar(decile_df['decile'], decile_df['observed_uplift'] * 100, color=colors, alpha=0.8, edgecolor='white')
ax.axhline(y=0, color='gray', linestyle='-', linewidth=1)

# Add value labels
for bar, (_, row) in zip(bars, decile_df.iterrows()):
    height = bar.get_height()
    label_y = height + 0.05 if height >= 0 else height - 0.15
    ax.text(bar.get_x() + bar.get_width()/2, label_y, f'{height:.2f}pp',
            ha='center', va='bottom' if height >= 0 else 'top', fontsize=10, fontweight='bold')

ax.set_xlabel('Uplift Decile (1=Lowest, 10=Highest)', fontsize=12)
ax.set_ylabel('Observed Uplift (percentage points)', fontsize=12)
ax.set_title('Marginal Observed Uplift by Decile — Decile 9 Dip Analysis', fontsize=14, fontweight='bold')
ax.set_xticks(decile_df['decile'])

# Annotate Decile 9
ax.annotate('Decile 9 DIP!\n"Sure Things" mixed in', xy=(9, decile_df[decile_df['decile']==9]['observed_uplift'].values[0]*100),
            xytext=(7, 1.0),
            arrowprops=dict(arrowstyle='->', color='red', lw=2),
            fontsize=11, color='red', fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='#ffcccc', alpha=0.8))

plt.tight_layout()
plt.savefig('../data/marginal_uplift_by_decile.png', dpi=150, bbox_inches='tight')
plt.show()

---

## 7. Summary: Phase 3 Key Findings

In [ ]:
print("PHASE 3 SUMMARY: MODEL EVALUATION")
print("=" * 70)
print(f"""
QINI CURVE RESULTS:
  • Model AUUC: {auuc:.4f}
  • Random AUUC: {auuc_random:.4f}
  • AUUC Gain: {auuc_gain:.4f}
  • Model is {(normalized_auuc-1)*100:.1f}% better than random targeting

DECILE ANALYSIS (Key Findings):
  • Best Decile: Decile 8 with +{decile_df[decile_df['decile']==8]['observed_uplift'].values[0]*100:.2f}pp observed lift
  • Decile 10: +{decile_df[decile_df['decile']==10]['observed_uplift'].values[0]*100:.2f}pp observed lift
  • Decile 9 ANOMALY: Only +{decile_df[decile_df['decile']==9]['observed_uplift'].values[0]*100:.2f}pp (model calibration issue)

DECILE 9 ROOT CAUSE:
  • Control rate: {decile_df[decile_df['decile']==9]['control_rate'].values[0]*100:.2f}% (highest of all deciles!)
  • These are "Sure Things" — high baseline buyers
  • Model confused high P(buy) with high persuadability
  
BUSINESS INSIGHT:
  ✅ Model successfully identifies Persuadables (Deciles 8, 10 have highest lift)
  ✅ Model beats random targeting by {(normalized_auuc-1)*100:.1f}%
  ⚠️ Calibration issue: Decile 9 contains misclassified "Sure Things"
  
INTERVIEW TALKING POINT:
  "My model identified Persuadables, but I also discovered a calibration gap:
  Decile 9 had high predicted uplift but low observed lift, because the model
  confused 'likely to buy' with 'persuadable.' This is a known limitation of
  T-Learners — they can struggle to separate base rates from treatment effects."

→ Phase 4: Business simulation to calculate ROI impact
""")